In [1]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [2]:
os.environ['hf_token'] = user_secrets.get_secret("HF_TOKEN")
os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_TOKEN")

In [3]:
assert(os.getenv('hf_token'))
assert(os.getenv('WANDB_API_KEY'))

In [4]:
%pip install peft wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [5]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import PrefixTuningConfig, get_peft_model
import wandb
from huggingface_hub import login
import os

2025-05-06 07:47:18.729168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746517638.923248      71 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746517638.979782      71 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
%pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3582bdadb6f6c5e4d17d643e4c79e3ece64d314587e9d7b98f7409f86279c549
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [7]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [8]:
from datetime import datetime
import pytz

now_utc = datetime.now(pytz.utc)
now_colombo = now_utc.astimezone(pytz.timezone('Asia/Colombo'))
run_name = now_colombo.strftime('%Y/%b/%d - %H:%M:%S')
print(run_name)

2025/May/06 - 13:17:39


In [9]:
wandb.login(key=os.getenv('WANDB_API_KEY'))
wandb.init(project="prefix-tuning-qwen-25-question-generation", name=run_name)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rtweera (rtw-rtweera) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [10]:
login(token=os.getenv('hf_token'))

In [11]:
dataset = load_dataset('squad')

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [12]:
import pandas as pd

In [13]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [15]:
from pprint import pprint

In [16]:
pprint(dataset['train'][234])

{'answers': {'answer_start': [74],
             'text': ['Michigan Wolverines football team']},
 'context': 'The Notre Dame football team has a long history, first beginning '
            'when the Michigan Wolverines football team brought football to '
            'Notre Dame in 1887 and played against a group of students. In the '
            'long history since then, 13 Fighting Irish teams have won '
            'consensus national championships (although the university only '
            'claims 11), along with another nine teams being named national '
            'champion by at least one source. Additionally, the program has '
            'the most members in the College Football Hall of Fame, is tied '
            'with Ohio State University with the most Heisman Trophies won, '
            'and have the highest winning percentage in NCAA history. With the '
            'long history, Notre Dame has accumulated many rivals, and its '
            'annual game against USC for the

In [17]:
model_name = 'Qwen/Qwen2.5-0.5B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [18]:
from datasets import Dataset, DatasetDict

In [19]:
def preprocess(example_batch):
    prompt = 'Generate a question from this given context:'
    inputs = [f'{prompt} {context} -> {question}' for context, question in zip(example_batch['context'], example_batch['question'])]
    tokenized = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    tokenized['labels'] = tokenized['input_ids'].clone()
    return tokenized
    

In [20]:
tokenized = dataset.map(preprocess, batched=True, remove_columns = dataset['train'].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [21]:
tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10570
    })
})

In [22]:
tokenized['validation'][0]

{'input_ids': [31115,
  264,
  3405,
  504,
  419,
  2661,
  2266,
  25,
  7297,
  20288,
  220,
  20,
  15,
  572,
  458,
  3693,
  8964,
  1809,
  311,
  8253,
  279,
  18319,
  315,
  279,
  5055,
  20761,
  8953,
  320,
  87101,
  8,
  369,
  279,
  220,
  17,
  15,
  16,
  20,
  3200,
  13,
  576,
  3693,
  20761,
  14872,
  320,
  32,
  6754,
  8,
  18319,
  22117,
  41594,
  23283,
  279,
  5055,
  20761,
  14872,
  320,
  45,
  6754,
  8,
  18319,
  12740,
  44167,
  220,
  17,
  19,
  4142,
  16,
  15,
  311,
  7232,
  862,
  4843,
  7297,
  20288,
  2265,
  13,
  576,
  1809,
  572,
  6342,
  389,
  7400,
  220,
  22,
  11,
  220,
  17,
  15,
  16,
  21,
  11,
  518,
  55041,
  594,
  22636,
  304,
  279,
  5836,
  12879,
  9154,
  12030,
  518,
  15993,
  50557,
  11,
  7043,
  13,
  1634,
  419,
  572,
  279,
  220,
  20,
  15,
  339,
  7297,
  20288,
  11,
  279,
  10734,
  45628,
  279,
  330,
  97235,
  21582,
  1,
  448,
  5257,
  6623,
  56589,
  27172,
  11,
  438,
  

In [23]:
train_set = tokenized['train'].select(range(1000)) 
eval_set = tokenized['validation'].select(range(40))

In [24]:
base_model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [25]:
# Function to generate questions
def generate_question(context, model, tokenizer, max_new_tokens=30):
    input_text = f"Generate a question: {context} -> "
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=256,
        truncation=True
    ).to(model.device)
    question_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id
    )
    question = tokenizer.decode(question_ids[0], skip_special_tokens=True)
    # Extract question after "->"
    question = question.split("->")[-1].strip()
    return question

In [26]:
# Evaluate base model
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
base_bleu_scores = []
base_rouge_scores = []

In [27]:
# Flatten validation dataset for evaluation
eval_contexts = []
eval_questions = []
for example in dataset["validation"]:
    eval_contexts.append(example['context'])
    eval_questions.append(example['question'])

In [28]:
from tqdm import tqdm

In [29]:
# Evaluate on subset
for i in tqdm(range(len(eval_set))):  # Match small_eval size
    context = eval_contexts[i]
    reference = eval_questions[i]
    question = generate_question(context, base_model, tokenizer)
    # BLEU score
    bleu_score = sentence_bleu(reference.split(), question.split())
    base_bleu_scores.append(bleu_score)
    # ROUGE-L score
    rouge_score = scorer.score(reference, question)["rougeL"].fmeasure
    base_rouge_scores.append(rouge_score)

 75%|███████▌  | 30/40 [00:29<00:09,  1.04it/s]/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram ord

In [30]:
# Compute average scores
base_bleu_avg = np.mean(base_bleu_scores)
base_rouge_avg = np.mean(base_rouge_scores)
print("Base Model Scores:", {"bleu": base_bleu_avg, "rougeL": base_rouge_avg})

# Log to W&B
wandb.log({"base_bleu": base_bleu_avg, "base_rougeL": base_rouge_avg})

Base Model Scores: {'bleu': 1.8717975852290112e-232, 'rougeL': 0.1343752489641756}


### Tuning part

In [ ]:
# Step 3: Configure Prefix Tuning
# Define prefix tuning configuration
peft_config = PrefixTuningConfig(
    task_type="CAUSAL_LM",
    num_virtual_tokens=20,  # with 20, you hit upto 17GB of RAM usage - apparently it is due to the savings of logits, oversome with predict_with_generate = True
    prefix_projection=True  # pass through an additional 2 layer MLP before attaching them to Key or Value - this enhances the expressiveness - hence more accurate. But more complex, risk of overfitting, more computational resources need, more time to train
)

In [32]:
# Load model and apply prefix tuning
model = AutoModelForCausalLM.from_pretrained(model_name)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Move to GPU if available
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

trainable params: 811,648 || all params: 494,844,416 || trainable%: 0.1640


In [33]:
tokenizer.eos_token_id

151643

In [34]:
# Step 4: Define Metrics and Training Arguments
# Compute BLEU and ROUGE metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # print('predictions type: ', type(predictions), 'shape: ', getattr(predictions, 'shape', 'N/A'))
    # print('labels type: ', type(labels), 'shape: ', getattr(labels, 'shape', 'N/A'))
    # O/P for above
    # predictions type: <class 'numpy.ndarray'> shape: (20, 256, 151936)
    # labels type: <class 'numpy.ndarray'> shape: (20, 256)

    # in classification we get a 2D predictions tensor (batch size, # of classes) e.g. [[0.1, 0.5], [-0.6, 1.5]] = two batches, each with two classes and logit values for each class as int
    # in LM, we get 3D predictions tensor (batch size, seq length, vocab size) e.g. [{(0.1, 0.5, ...to vocab size), (-0.6, 1.5, ...to vocab size), ...all sequences}, {next item in batch}]

    predictions = predictions.argmax(-1)    # argmax of last axis (i.e. logits axis)
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)   # labels where it should be ignored in loss (e.g. padding) are set to -100, which the tokenizer can't decode

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Extract question part after "->"
    decoded_preds = [pred.split("->")[-1].strip() if "->" in pred else pred for pred in decoded_preds]
    decoded_labels = [label.split("->")[-1].strip() if "->" in label else label for label in decoded_labels]
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    bleu_scores = []
    rouge_scores = []
    for pred, ref in zip(decoded_preds, decoded_labels):
        bleu_scores.append(sentence_bleu([ref.split()], pred.split()))
        rouge_scores.append(scorer.score(ref, pred)["rougeL"].fmeasure)
    return {
        "bleu": np.mean(bleu_scores),
        "rougeL": np.mean(rouge_scores)
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="./qwen25-0.5b-prefix-tuning",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,
    eval_accumulation_steps=1,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_steps=10,
    logging_first_step=True,
    eval_on_start=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,   # dont set this for PEFT methods, causees errors, manually load the best model
    push_to_hub=True,
    report_to="wandb",
    run_name=f'{run_name}-prefix-tuned',
)

In [36]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=eval_set,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [38]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Rougel
1,1.682300,2.563648,0.001705,0.070086
2,1.562900,2.386212,0.002233,0.071315
3,1.401200,2.293324,0.002462,0.073889
4,1.467600,2.232627,0.003662,0.072380
5,1.412800,2.195016,0.004479,0.072105
6,1.469000,2.171984,0.004458,0.072721
7,1.433700,2.162472,0.004458,0.072636
8,1.383300,2.158943,0.004458,0.072662


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis co

TrainOutput(global_step=1000, training_loss=1.4887142494916916, metrics={'train_runtime': 981.8586, 'train_samples_per_second': 8.148, 'train_steps_per_second': 1.018, 'total_flos': 4397852000256000.0, 'train_loss': 1.4887142494916916, 'epoch': 8.0})

In [39]:
# Evaluate prefix-tuned model
eval_results = trainer.evaluate()
print("Prefix-Tuned Model Scores:", eval_results)

# Log to W&B
wandb.log({
    "prefix_bleu": eval_results["eval_bleu"],
    "prefix_rougeL": eval_results["eval_rougeL"]
})


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Prefix-Tuned Model Scores: {'eval_loss': 2.1589431762695312, 'eval_bleu': 0.00445772802493562, 'eval_rougeL': 0.07266155761421292, 'eval_runtime': 36.5133, 'eval_samples_per_second': 1.095, 'eval_steps_per_second': 0.548, 'epoch': 8.0}


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [40]:
# Compare with base model
print("\nComparison:")
print(f"Base Model - BLEU: {base_bleu_avg:.4f}, ROUGE-L: {base_rouge_avg:.4f}")
print(f"Prefix-Tuned - BLEU: {eval_results['eval_bleu']:.4f}, ROUGE-L: {eval_results['eval_rougeL']:.4f}")


Comparison:
Base Model - BLEU: 0.0000, ROUGE-L: 0.1344
Prefix-Tuned - BLEU: 0.0045, ROUGE-L: 0.0727


In [41]:
# Push model to Hugging Face Hub
trainer.push_to_hub("qwen25-0.5b-prefix-tuning-question-generation")

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rtweera/qwen25-0.5b-prefix-tuning/commit/c1963aa82ab32e2995439b7c1d8e83f0d5157f28', commit_message='qwen25-0.5b-prefix-tuning-question-generation', commit_description='', oid='c1963aa82ab32e2995439b7c1d8e83f0d5157f28', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rtweera/qwen25-0.5b-prefix-tuning', endpoint='https://huggingface.co', repo_type='model', repo_id='rtweera/qwen25-0.5b-prefix-tuning'), pr_revision=None, pr_num=None)

In [42]:
# Finish W&B run
wandb.finish()

base_bleu,▁
base_rougeL,▁
eval/bleu,▁▂▃▆█████
eval/loss,█▅▃▂▂▁▁▁▁
eval/rougeL,▁▃█▅▅▆▆▆▆
eval/runtime,█▂▂▁▁▁▁▁▁
eval/samples_per_second,▁▇▇██████
eval/steps_per_second,▁▇▇▇▇████
prefix_bleu,▁
prefix_rougeL,▁
train/epoch,▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██████
